In [ ]:
###########################################
##                  TFT                  ##
###########################################
## Chargement des données
## Conversion des types
## Séparation Train / Test
## Préparation des TimeSeriesDataSet
## Définition des DataLoaders
## Initialisation du modèle TFT
## Entraînement du modèle
## Sauvegarde et chargement du modèle
## Prédiction et affichage des résultats

In [1]:
import pandas as pd
import time
import torch
from pytorch_forecasting import TimeSeriesDataSet
from pytorch_forecasting.metrics import QuantileLoss
from pytorch_forecasting.data.encoders import NaNLabelEncoder
from pytorch_lightning import Trainer
from torch.utils.data import DataLoader
from pytorch_lightning.callbacks import EarlyStopping

# Charger le dataset
df = pd.read_csv(r"..\..\..\..\Datasources\MetroPT3_new_imputed_final.csv", delimiter=",", decimal=".", index_col=0)
df.reset_index(drop=True, inplace=True)
display(df.head(2))

# Dataset commence le 2020-04-12 11:20:00 et se termine le 2020-07-17 06:00:00
pannes = [
    {'id': 'Panne1',  'start': '2020-04-12 11:50:00', 'end': '2020-04-12 23:30:00'},
    {'id': 'Panne2',  'start': '2020-04-18 00:00:00', 'end': '2020-04-18 23:59:00'},
    {'id': 'Panne3',  'start': '2020-04-19 00:00:00', 'end': '2020-04-19 01:30:00'},
    {'id': 'Panne4',  'start': '2020-04-29 03:20:00', 'end': '2020-04-29 04:00:00'},
    {'id': 'Panne5',  'start': '2020-04-29 22:00:00', 'end': '2020-04-29 22:20:00'},
    {'id': 'Panne6',  'start': '2020-05-13 14:00:00', 'end': '2020-05-13 23:59:00'},
    {'id': 'Panne7',  'start': '2020-05-18 05:00:00', 'end': '2020-05-18 05:30:00'},
    {'id': 'Panne8',  'start': '2020-05-19 10:10:00', 'end': '2020-05-19 11:00:00'},
    {'id': 'Panne9',  'start': '2020-05-19 22:10:00', 'end': '2020-05-19 23:59:00'},
    {'id': 'Panne10', 'start': '2020-05-20 00:00:00', 'end': '2020-05-20 20:00:00'},
    {'id': 'Panne11', 'start': '2020-05-23 09:50:00', 'end': '2020-05-23 10:10:00'},
    {'id': 'Panne12', 'start': '2020-05-29 23:30:00', 'end': '2020-05-29 23:59:00'},
    {'id': 'Panne13', 'start': '2020-05-30 00:00:00', 'end': '2020-05-30 06:00:00'},
    {'id': 'Panne14', 'start': '2020-06-01 15:00:00', 'end': '2020-06-01 15:40:00'},
    {'id': 'Panne15', 'start': '2020-06-03 10:00:00', 'end': '2020-06-03 11:00:00'},
    {'id': 'Panne16', 'start': '2020-06-05 10:00:00', 'end': '2020-06-05 23:59:00'},
    {'id': 'Panne17', 'start': '2020-06-06 00:00:00', 'end': '2020-06-06 23:59:00'},
    {'id': 'Panne18', 'start': '2020-06-07 00:00:00', 'end': '2020-06-07 14:30:00'},
    {'id': 'Panne19', 'start': '2020-07-08 17:30:00', 'end': '2020-07-08 19:00:00'},
    {'id': 'Panne20', 'start': '2020-07-15 14:30:00', 'end': '2020-07-15 19:00:00'},
    {'id': 'Panne21', 'start': '2020-07-17 04:30:00', 'end': '2020-07-17 05:30:00'}
         ]



,timestamp,TP2,TP3,H1,DV_pressure,Reservoirs,Oil_temperature,Motor_current,COMP,DV_eletric,Towers,MPG,LPS,Pressure_switch,Oil_level,Caudal_impulses,panne
0,2020-04-12 11:20:00,9.128,8.650,-0.018,-0.022,8.648,55.475,6.0175,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0
1,2020-04-12 11:20:10,9.354,8.896,-0.016,-0.020,8.892,56.525,6.0075,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0


In [2]:

# Convertir timestamp en datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Définir la date de séparation entre Train et Test
split_date = "2020-06-05 23:59:10"  # Séparation temporelle (Fin de la panne 16)

# Séparer les données en train/test
df_train = df[df["timestamp"] < split_date].copy()
df_test = df[df["timestamp"] >= split_date].copy()

# Recalculer time_idx en divisant par 10 pour éviter les grands nombres
df_train["time_idx"] = ((df_train["timestamp"] - df_train["timestamp"].min()).dt.total_seconds() // 10).astype(int)
df_test["time_idx"] = ((df_test["timestamp"] - df_test["timestamp"].min()).dt.total_seconds() // 10).astype(int)

print(f"Min time_idx in df_train: {df_train['time_idx'].min()}, Max: {df_train['time_idx'].max()}")
print(f"Min time_idx in df_test: {df_test['time_idx'].min()}, Max: {df_test['time_idx'].max()}")


# Définir les colonnes catégoriques et les forcer en str avant de les convertir en category
for col in ["COMP", "DV_eletric", "Towers"]:
    df_train[col] = df_train[col].astype(int).astype(str).astype("category")
    df_test[col] = df_test[col].astype(int).astype(str).astype("category")

# Conversion explicite de panne
df_train["panne"] = df_train["panne"].astype(int).astype(str).astype("category")
df_test["panne"] = df_test["panne"].astype(int).astype(str).astype("category")

print("Conversion des colonnes catégoriques et de panne terminée.")

# Ajouter un group_id unique pour tout le dataset
df_train["group_id"] = df_train["COMP"].astype(str) + "_" + (df_train["time_idx"] // 1000).astype(str)
df_test["group_id"] = df_test["COMP"].astype(str) + "_" + (df_test["time_idx"] // 1000).astype(str)

print("group_id ajouté.")

# Définir les encodeurs pour les variables catégoriques
categorical_encoders = {
    "COMP": NaNLabelEncoder(),
    "DV_eletric": NaNLabelEncoder(),
    "Towers": NaNLabelEncoder(),
}

print("Encodeurs catégoriques définis.")

Min time_idx in df_train: 0, Max: 471114
Min time_idx in df_test: 0, Max: 356405
Conversion des colonnes catégoriques et de panne terminée.
group_id ajouté.
Encodeurs catégoriques définis.


In [3]:

# Définir l'horizon de prévision et la fenêtre d'observation
max_encoder_length = 30  # Fenêtre plus longue pour éviter la suppression
max_prediction_length = 3  # Une seule prédiction pour éviter les erreurs

train_dataset = TimeSeriesDataSet(
    df_train,
    time_idx="time_idx",
    target="panne",
    group_ids=["group_id"],  # Utiliser un ID unique par série
    time_varying_known_reals=["TP2", "H1", "DV_pressure", "Oil_temperature", "Motor_current"],
    time_varying_known_categoricals=["DV_eletric", "Towers"],
    max_encoder_length=30,  
    max_prediction_length=3,  
    min_encoder_length=10,  # Augmenter pour éviter des séquences trop courtes
    min_prediction_length=3,
    categorical_encoders=categorical_encoders,
    allow_missing_timesteps=True
)


print(f"Dataset train créé avec {len(train_dataset)} séquences.")

test_dataset = TimeSeriesDataSet(
    df_test,
    time_idx="time_idx",
    target="panne",
    group_ids=["COMP"],
    time_varying_known_reals=["TP2", "H1", "DV_pressure", "Oil_temperature", "Motor_current"],
    time_varying_known_categoricals=["DV_eletric", "Towers"],
    max_encoder_length=10,  # Doit être identique à train_dataset
    max_prediction_length=3,  # Doit être identique à train_dataset
    min_encoder_length=5,
    min_prediction_length=2,
    categorical_encoders=categorical_encoders,
    allow_missing_timesteps=True
)


# Correction des noms de clés pour correspondre à ceux attendus par le modèle TFT
def fix_dataset_structure(dataset):
    for i in range(len(dataset)):
        if "encoder_length" in dataset[i][0]:
            dataset[i][0]["encoder_lengths"] = dataset[i][0].pop("encoder_length")
        if "decoder_length" in dataset[i][0]:
            dataset[i][0]["decoder_lengths"] = dataset[i][0].pop("decoder_length")

# Appliquer la correction aux datasets
fix_dataset_structure(train_dataset)
fix_dataset_structure(test_dataset)

print(f" Dataset train corrigé avec {len(train_dataset)} séquences.")
print(f" Dataset test corrigé avec {len(test_dataset)} séquences.")


C:\Users\usermine\Amine-Jupiter\SESSION6\ProjetSynthese\Modele1_Classification\TFT\tft_env\Lib\site-packages\pytorch_forecasting\data\timeseries.py:1301: UserWarning: Min encoder length and/or min_prediction_idx and/or min prediction length and/or lags are too large for 12 series/groups which therefore are not present in the dataset index. This means no predictions can be made for those series. First 10 removed groups: [{'__group_id__group_id': '0_131'}, {'__group_id__group_id': '0_147'}, {'__group_id__group_id': '0_205'}, {'__group_id__group_id': '0_276'}, {'__group_id__group_id': '0_286'}, {'__group_id__group_id': '0_41'}, {'__group_id__group_id': '0_77'}, {'__group_id__group_id': '0_83'}, {'__group_id__group_id': '0_84'}, {'__group_id__group_id': '0_85'}]
  warnings.warn(


Dataset train créé avec 491829 séquences.
 Dataset train corrigé avec 491829 séquences.
 Dataset test corrigé avec 357868 séquences.


In [4]:
from pytorch_forecasting.models import TemporalFusionTransformer
from pytorch_forecasting.metrics import QuantileLoss

# Définition du modèle TFT (Pas besoin de `LightningModule`)
tft = TemporalFusionTransformer.from_dataset(
    train_dataset,
    learning_rate=1e-3,
    hidden_size=16,
    attention_head_size=4,
    dropout=0.1,
    hidden_continuous_size=8,
    loss=QuantileLoss(),
    log_interval=10,
    reduce_on_plateau_patience=4,
)

# Pas besoin d'assertion sur `LightningModule`
print(f"Modèle TFT initialisé avec {tft.size()} paramètres.")

C:\Users\usermine\Amine-Jupiter\SESSION6\ProjetSynthese\Modele1_Classification\TFT\tft_env\Lib\site-packages\lightning\pytorch\utilities\parsing.py:209: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
C:\Users\usermine\Amine-Jupiter\SESSION6\ProjetSynthese\Modele1_Classification\TFT\tft_env\Lib\site-packages\lightning\pytorch\utilities\parsing.py:209: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.


Modèle TFT initialisé avec 21007 paramètres.


C:\Users\usermine\Amine-Jupiter\SESSION6\ProjetSynthese\Modele1_Classification\TFT\tft_env\Lib\site-packages\pytorch_forecasting\models\temporal_fusion_transformer\__init__.py:171: UserWarning: In pytorch-forecasting models, on versions 1.1.X, the default optimizer defaults to 'adam', if pytorch_optimizer is not installed, otherwise it defaults to 'ranger' from pytorch_optimizer. From version 1.2.0, the default optimizer will be 'adam' regardless of whether pytorch_optimizer is installed, in order to minimize the number of dependencies in default parameter settings. Users who wish to ensure their code continues using 'ranger' as optimizer should ensure that pytorch_optimizer is installed, and set the optimizer parameter explicitly to 'ranger'.
  super().__init__(loss=loss, logging_metrics=logging_metrics, **kwargs)


In [5]:
from pytorch_lightning import LightningModule

# Création du wrapper LightningModule
class TFTLightningModule(LightningModule):
    def __init__(self, tft):
        super().__init__()
        self.model = tft

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_pred = self.model(x)
        loss = self.model.loss(y_pred, y)
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        if batch is None or "x" not in batch or "y" not in batch:
            print(f"Batch {batch_idx} mal structuré : {batch}")
            return None  # Évite une erreur bloquante

        x = batch["x"]
        y = batch["y"]

        try:
            y_pred = self.model(x)
            loss = self.model.loss(y_pred, y)
        except KeyError as e:
            print(f"Erreur de clé dans `validation_step`: {e}")
            return None

        return {"val_loss": loss}


    def configure_optimizers(self):
        return self.model.configure_optimizers()

# Encapsulation du modèle TFT dans `LightningModule`
tft_lightning = TFTLightningModule(tft)

In [6]:
import torch
from pytorch_lightning.callbacks import EarlyStopping, TQDMProgressBar
from pytorch_lightning import Trainer
from torch.utils.data import DataLoader

# Fonction collate corrigée
import torch
import numpy as np

def tft_collate_fn(batch):
    """ Collate function pour le modèle TFT, en s'assurant que les tailles et types sont cohérents. """
    
    batch = [b for b in batch if b is not None]  
    if not batch:
        return None  # Éviter une erreur si le batch est vide

    try:
        feature_dicts = [b[0] for b in batch]
        labels = [b[1][0] for b in batch]  

        # ✅ Correction des noms des clés
        required_keys = ["encoder_length", "decoder_length", "x_cat", "x_cont"]
        missing_keys = [k for k in required_keys if k not in feature_dicts[0]]
        if missing_keys:
            raise KeyError(f"Clés manquantes : {missing_keys}")

        # ✅ Trouver la séquence la plus longue
        max_seq_len = max(d["x_cat"].shape[0] for d in feature_dicts)

        def pad_tensor(tensor, max_len):
            """ Fonction pour padder les tenseurs à la même longueur """
            pad_size = max_len - tensor.shape[0]
            if pad_size > 0:
                pad = torch.zeros((pad_size, *tensor.shape[1:]), dtype=tensor.dtype)
                return torch.cat([tensor, pad], dim=0)
            return tensor

        # ✅ Appliquer le padding aux séquences pour uniformiser la taille
        batch_data = {
            k: torch.stack([pad_tensor(d[k], max_seq_len) for d in feature_dicts])
            for k in ["x_cat", "x_cont"]
        }

        # ✅ Empiler `encoder_length` et `decoder_length`
        batch_data["encoder_length"] = torch.tensor([d["encoder_length"] for d in feature_dicts])
        batch_data["decoder_length"] = torch.tensor([d["decoder_length"] for d in feature_dicts])

        # ✅ Conversion des labels en tenseur
        batch_labels = torch.stack([torch.tensor(label) for label in labels])

        return batch_data, batch_labels

    except Exception as e:
        print(f"Erreur `tft_collate_fn`: {e}")
        return None  


# Vérification du dataset
try:
    sample = train_dataset[0]
    print(f"Échantillon dataset : {list(sample[0].keys())}")
except Exception as e:
    print(f"Erreur lors du chargement du dataset : {e}")

# Configuration des DataLoaders
batch_size = 64
num_workers = 0  #Éviter multiprocessing sous Windows
pin_memory = torch.cuda.is_available()

train_dataloader = DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, drop_last=False, 
    collate_fn=tft_collate_fn, pin_memory=pin_memory
)

test_dataloader = DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, drop_last=False, 
    collate_fn=tft_collate_fn, pin_memory=pin_memory
)

# Vérification du premier batch
try:
    test_batch = next(iter(train_dataloader))
    if isinstance(test_batch, tuple) or test_batch is None:
        print(f" Erreur : Batch mal structuré ! Type : {type(test_batch)}")
    else:
        print(f" Batch structuré : Clés = {list(test_batch.keys())}")
except Exception as e:
    print(f" Erreur lors du chargement du batch : {e}")

# Callbacks pour l'entraînement
early_stop_callback = EarlyStopping(monitor="val_loss", patience=5, verbose=True, mode="min")
progress_bar = TQDMProgressBar(refresh_rate=10)

# Définition de l'entraîneur
trainer = Trainer(
    max_epochs=30,
    accelerator="auto",
    enable_progress_bar=True,
    enable_checkpointing=False,  
    callbacks=[early_stop_callback, progress_bar],  
    gradient_clip_val=0.1
)

# Lancement de l'entraînement
trainer.fit(tft_lightning, train_dataloader, test_dataloader)

print(" Entraînement terminé avec succès !")


C:\Users\usermine\AppData\Local\Temp\ipykernel_8692\3057267033.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch_labels = torch.stack([torch.tensor(label) for label in labels])
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Échantillon dataset : ['x_cat', 'x_cont', 'encoder_length', 'decoder_length', 'encoder_target', 'encoder_time_idx_start', 'groups', 'target_scale']
 Erreur : Batch mal structuré ! Type : <class 'tuple'>



  | Name  | Type                      | Params | Mode 
------------------------------------------------------------
0 | model | TemporalFusionTransformer | 21.0 K | train
------------------------------------------------------------
21.0 K    Trainable params
0         Non-trainable params
21.0 K    Total params
0.084     Total estimated model params size (MB)
326       Modules in train mode
0         Modules in eval mode


Sanity Checking: |                                                                               | 0/? [00:00<…

Batch 0 mal structuré : ({'x_cat': tensor([[[1, 1],
         [1, 1],
         [1, 1],
         ...,
         [1, 1],
         [1, 1],
         [1, 1]],

        [[1, 1],
         [1, 1],
         [1, 1],
         ...,
         [1, 1],
         [1, 1],
         [1, 1]],

        [[1, 1],
         [1, 1],
         [1, 0],
         ...,
         [1, 1],
         [1, 1],
         [1, 1]],

        ...,

        [[1, 1],
         [1, 1],
         [1, 0],
         ...,
         [1, 1],
         [1, 1],
         [1, 1]],

        [[1, 1],
         [1, 0],
         [1, 0],
         ...,
         [1, 1],
         [1, 1],
         [1, 1]],

        [[1, 0],
         [1, 0],
         [1, 0],
         ...,
         [1, 1],
         [1, 1],
         [1, 0]]]), 'x_cont': tensor([[[ 1.0000,  1.8981, -1.9854,  5.4273,  1.6739,  1.3696],
         [ 1.0000,  1.9047, -1.9854,  5.4627,  1.6668,  1.3602],
         [ 1.0000,  1.9052, -1.9854,  5.4627,  1.6456,  1.3249],
         ...,
         [ 1.0000,  1.2

C:\Users\usermine\Amine-Jupiter\SESSION6\ProjetSynthese\Modele1_Classification\TFT\tft_env\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.
C:\Users\usermine\AppData\Local\Temp\ipykernel_8692\3057267033.py:49: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch_labels = torch.stack([torch.tensor(label) for label in labels])
C:\Users\usermine\Amine-Jupiter\SESSION6\ProjetSynthese\Modele1_Classification\TFT\tft_env\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_worker

Training: |                                                                                      | 0/? [00:00<…

KeyError: 'encoder_lengths'

In [ ]:
##############################################
## 🔟 Sauvegarde et chargement du modèle    ##
##############################################

model_path = r"..\..\Generated_Files\TFT\tft_model.ckpt"
trainer.save_checkpoint(model_path)
print("Modèle sauvegardé.")

tft_loaded = TemporalFusionTransformer.load_from_checkpoint(model_path)
print("Modèle chargé avec succès.")

In [ ]:
##############################################
## 1️⃣1️⃣ Prédictions avec le modèle       ##
##############################################

predictions = tft.predict(test_dataloader, mode="prediction")

print("Prédictions :", predictions[:10])